## 2.3 Tensor culculation

In [2]:
import numpy as np

### Unfolding (matricization)
def unfold(X, mode_number):
    """
    Unfolding (matricization)

    Parameters
    ----------
    X : ndarray
        Input tensor.
    mode_number : int
        Mode number along which to unfold.
        ``numpy's axis = (mode_number - 1)``

    Returns
    -------
    Y : ndarray
        Unfolded (Matrixed) tensor.
    """
    axis = mode_number - 1     # numpy's axis = (mode_number - 1)
    N = np.ndim(X)             # order of input tensor
    L = np.size(X)             # total number of input tensor elements
    col_len = np.size(X, axis) # column Length of output matrix
    row_len = int(L / col_len) # row Length of output matrix
    ### mode (axis) transpose
    permute = [axis] + list(range(0, axis)) + list(range(axis+1, N))
    X_transposed = np.transpose(X, permute)
    ### unfolding 
    unfolding_shape = (col_len, row_len)
    Y = np.reshape(X_transposed, unfolding_shape)
    return Y

### Folding (tensorization)
def fold(Y, mode_number, shape):
    """
    Unfolding (matricization)

    Parameters
    ----------
    Y : ndarray
        Input matrix.
    mode_number : int
        Mode number along which to fold.
        ``numpy's axis = (mode_number - 1)``
    shape : tuple or list of ints
        shape of folding (tensorization).

    Returns
    -------
    X : ndarray
        Folded (tensorized) matrix.
    """
    axis = mode_number - 1 # numpy's axis = (mode_number - 1)
    N = len(shape)         # order of output tensor
    ### folding
    permute = [axis] + list(range(0, axis)) + list(range(axis+1, N))
    folding_shape = []
    for i in permute: folding_shape.append(shape[i])
    Y_folded = np.reshape(Y, folding_shape)
    ### mode (axis) transpose
    if mode_number==1: permute_inv = list(range(0, N))
    else: permute_inv = list(range(1, axis+1)) + [0] + list(range(axis+1, N))
    X = np.transpose(Y_folded, permute_inv)
    return X

### 2.3.9.1 $n$-th mode product. mode-$n$ product

In [50]:
### n-th mode product of Tensor and Matrix
def n_mode_prod_TandM(Tensor, Matrix, mode_number):
    """
    n-th mode product of Tensor and Matrix

    Parameters
    ----------
    Tensor : ndarray
        Input tensor.
    Matrix : ndarray
        Input matrix.
    mode_number : int
        Mode number to calculate the product.
        ``numpy's axis = (mode_number - 1)``

    Returns
    -------
    Y : ndarray
        Producted tensor.
    """

    axis = mode_number - 1          # numpy's axis = (mode_number - 1)
    tensor_shape = np.shape(Tensor) # tensor's shape

    ### new tensor's shape
    new_tensor_shape = np.copy(tensor_shape)
    new_tensor_shape[axis] = np.size(Matrix, 0)

    ### unfolding
    Tensor_unfolded = unfold(Tensor, mode_number)

    ### Matrix product of Tensor and Matrix
    matrix_product_TandM = np.matmul(Matrix, Tensor_unfolded)

    ### Folding
    Y = fold(matrix_product_TandM, mode_number, new_tensor_shape)

    return Y

### Example
mode_number = 3
A = np.random.random((2, 3))
X = np.random.random((2, 4, 3))
Y = n_mode_prod_TandM(X, A, mode_number)
print(str(mode_number)+"th-mode prodect")
print("``Y = X x A``")
print("Tensor X shape "+str(X.shape))
print("Matrix A shape "+str(A.shape))
print("Tensor Y shape "+str(Y.shape))

3th-mode prodect
``Y = X x A``
Tensor X shape (2, 4, 3)
Matrix A shape (2, 3)
Tensor Y shape (2, 4, 2)


### 2.3.9.2 all-mode product

In [61]:
### all-mode product of Tensor and Matrix
def all_mode_prod_TandM(Tensor, Matrix_list):
    """
    All mode product of Tensor and Matrix

    Parameters
    ----------
    Tensor : ndarray
        Input tensor.
    Matrix_list : list ``[ndarray, ndarray, ndarray,...]``
        Input matrices list.

    Returns
    -------
    Y : ndarray
        Producted tensor.
    """

    ### all-mode product of Tensor and Matrix
    Y = np.copy(Tensor)
    for axis, Matrix in enumerate(Matrix_list):
        ### n-th mode product of Tensor and Matrix
        mode_number = axis + 1
        Y = n_mode_prod_TandM(Y, Matrix, mode_number)

    return Y

### Example
A_list = []
A_list.append(np.random.random((10, 3)))
A_list.append(np.random.random((20, 4)))
A_list.append(np.random.random((30, 5)))
G = np.random.random((3, 4, 5))
Y = all_mode_prod_TandM(G, A_list)
print("all-mode prodect")
print("``Y = G x [A]``")
print("Tensor  G   shape "+str(G.shape))
for i, A in enumerate(A_list): print("Matrix X["+str(i+1)+"] shape "+str(A.shape))
print("Tensor  Y   shape "+str(Y.shape))

all-mode prodect
``Y = G x [A]``
Tensor  G   shape (3, 4, 5)
Matrix X[1] shape (10, 3)
Matrix X[2] shape (20, 4)
Matrix X[3] shape (30, 5)
Tensor  Y   shape (10, 20, 30)
